# スタイン推定量を確かめてみる
- Y.Nakahashi
- 2018-01-25

[ベイズモデリングの世界](https://www.iwanami.co.jp/book/b341709.html)を読んでいて非常に面白い話題を発見しました。P118から始まる**スタイン推定量**についてなのですが、その直感的な意味（本文から引用）：

> それぞれの$y_i$を、全部の{$y_i$}の平均値$\frac{1}{n}\Sigma_{i=1}^{n}y_i$の方向に$a$だけ引っ張ってやる

ことで、$y_i$そのものを推定量とするよりも、パラメータの真値$\hat{\theta_{i}}$との二乗誤差の期待値が小さくなるそうです。これはいわゆる縮小推定量の一種で、引っ張りの程度$a$をデータから適応的に求めるのがこの推定量のキモなのだそうです。

スタイン推定量$\hat{\theta_{i}^{S}}$の定義自体はそれほど難しいものでもなく、以下のような式によって表されます：

$$
\begin{equation*}
\hat{\theta_{i}^{S}} = (1-a)y_{i} + \frac{n}{a}\Sigma_{i=1}^{n}y_i \\
\end{equation*}
$$
ここで、
$$
\begin{equation*}
a = \frac{\sigma^2}{s^2},  \\
s^2 = \frac{1}{n-3}\Sigma_{i=1}^{n}(y_i - \frac{1}{n}\Sigma_{j=1}^{n}y_i)^2
\end{equation*}
$$
です。このスタイン推定量が本当に$y_i$よりも二乗誤差が小さくなるのか、以下のように実験してみます。


#### 実験内容
実験内容は非常に簡単で、平均と分散が共に既知である正規分布からいくつかのサンプルを生成します。そのサンプルをそのまま用いた場合と、スタイン推定量として用いた場合とで、真のパラメータとの二乗誤差：

$$
\begin{equation*}
\mathbb{E}[\Sigma_{i=1}^{n}(\theta_{i}^{*}(\{y_i\}) - \theta_i)^2]
\end{equation*}
$$

にどれほど差が生じるかを確認しています。

#### パラメータ設定
実験では、一回の試行につき平均が{1，2，...，n}、分散が1である正規分布に従うサンプルをn個生成することとしました。このn個のデータから各サンプルのスタイン推定量を求め、真値である{1，2，...，n}との二乗誤差を計算します。同時にデータそのものを用いた場合の二乗誤差も計算しておきます。上記の実験を1000回繰り返し、最終的にどちらが小さい値となるかを比較しました。

In [1]:
n   <- 10
mu  <- 1:n
sig <- 1

K   <- 1000
res <- matrix(0, K, 2)

#### 実験開始
上記のパラメータに基づき、以下のように計算を行います。

In [2]:
set.seed(123)
for (i in 1:K) {
   ## サンプルを発生させる
   tmp <- rnorm(n, mu, sig)
   
   ## スタイン推定値を求める
   s2  <- (1/(n-3)) * sum((tmp - mean(tmp))^2)      
   a   <- sig / s2
   s_i <- (1-a)*tmp + (a/n)*sum(tmp)

   ## 結果を保存する
   res[i, 1] <- sum((s_i - mu)^2)/n # スタイン推定値を用いた二乗誤差の期待値
   res[i, 2] <- sum((tmp - mu)^2)/2 # データそのものを用いた二条誤差の期待値
}

#### 結果
それでは結果です。

In [3]:
mean(res[, 1] / res[, 2])
sum(res[, 1] > res[, 2])

[1] 0.1923771

[1] 0

スタイン推定量を用いた場合、データそのものの場合と比較して二乗誤差が20%（80%減！）と劇的に小さくなっています。しかも1000回の試行において、スタイン推定量がデータを用いた場合よりも二乗誤差で大きくなったことは一度もない、という結果となりました。
俄かに信じがたいのですが、かなり強烈な結果ですね。。。

#### 終わりに
今回の実験結果はなかなか衝撃的とも言えるものでしたが、実際のところ`n`が小さければここまで劇的な結果は生じないようです。理論的な展開が書籍にありますので詳細はそこに譲るとして、これの何が面白いかと言うと階層ベイズモデルとの関連性についてなんですね。現在、別の記事の内容として階層ベイズモデルに触れているのですが、その説明として

> パラメータを生成する分布を仮定することで、パラメータに緩やかな縛りをかける

といった言い回しを考えています。ここでふと「*階層ベイズでは（緩やかな縛りをかけることで）推定値にバイアスを与えているわけだが、それにも関わらずモデルとして良くなるというのは、一体どういう意味なのだろう？*」という疑問が浮かんできたのですが、その時にこのスタイン推定量のことを思い出しました。つまり、**階層ベイズではパラメータに対する分布を仮定することで推定値にバイアスを与えているが、全体的に見れば誤差を小さくしている**ということではないかと[^1]。

[^1]:なお「ベイズモデリングの世界」ではこれをバイアス-バリアンスのトレードオフとして解説しています。